In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [32]:
import sys
sys.setrecursionlimit(10 ** 5)
from collections import defaultdict

N = int(input())
C = [input() for _ in range(N)]

G = [list() for _ in range(2*N + 2)]
GT = [defaultdict(int) for _ in range(2 * N + 2)]
GB = [defaultdict(int) for _ in range(2 * N + 2)]

for i in range(N):
    G[0].append(i+1)
    GT[0][i+1] = 1
    G[N+i+1].append(2*N+1)
    GT[N+i+1][2*N+1] = 1

for i in range(N):
    for j in range(N):
        if C[i][j] == ".":
            continue
        G[i+1].append(N+j+1)
        G[N+j+1].append(i+1)
        GT[i+1][N+j+1] = 1
        GB[N+j+1][i+1] = 0
    
ans = 0
while 1:
    flow = 1
    v = [False] * (2 * N + 2)
    
    def dfs(x):
        global flow, ans
        
        v[x] = True
        if v[2 * N + 1]:
            ans += 1
            return
        
        for nex in G[x]:
            if v[nex]:
                continue
            
            if GT[x][nex]:
                dfs(nex)
                if v[2 * N + 1]:
                    GT[x][nex] -= flow
                    GB[nex][x] += flow
                    return
                
            if GB[x][nex]:
                dfs(nex)
                if v[2 * N + 1]:
                    GB[x][nex] -= flow
                    GT[nex][x] += flow
                    return
        return
    
    dfs(0)
    if not v[2 * N + 1]:
        break
        
print(ans)

149
